In [5]:
#simple linear regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
import time

%matplotlib qt

def load_dbscore_data():
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment2')
    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from db_score"
    curs.execute(sql)
    
    data  = curs.fetchall()
    
    curs.close()
    conn.close()
    
    #X = [ (t['attendance'], t['homework'], t['midterm'] ) for t in data ]
    X = [ ( t['midterm'] ) for t in data ]
    X = np.array(X)
    
    y = [ (t['score']) for t in data]
    y = np.array(y)

    return X, y

X, y = load_dbscore_data()

# y = mx + c

from matplotlib import pyplot as plt
from celluloid import Camera
fig = plt.figure()
ax = fig.subplots()

camera = Camera(fig)
x = np.linspace(0, 2 * np.pi)

import statsmodels.api as sm
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const)
ls = model.fit()

print(ls.summary())


ls_c = ls.params[0]
ls_m = ls.params[1]

c=ls_c
m=ls_m

y_pred = m*X + c

#plt.scatter(X, y) 
#plt.plot([min(X), max(X)], [min(y_pred), max(y_pred)], color='red')
#plt.show()

def gradient_descent_naive(X, y):

    epochs = 100000
    min_grad = 0.0001
    learning_rate = 0.001
    
    m = 0.0
    c = 0.0
    
    n = len(y)
    
    c_grad = 0.0
    m_grad = 0.0
    
    for epoch in range(epochs):
        
        for i in range(n):
            y_pred = m * X[i] + c
            m_grad += 2*(y_pred-y[i]) * X[i]
            c_grad += 2*(y_pred - y[i])

        c_grad /= n
        m_grad /= n
        
        m = m - learning_rate * m_grad
        c = c - learning_rate * c_grad
        
        y_pred /= n
        
        if ( epoch % 1000 == 0):
            print("epoch %d: m_grad=%f, c_grad=%f, m=%f, c=%f" %(epoch, m_grad, c_grad, m, c) ) 
            
            ax.scatter(X, y, color='blue') 
            ax.plot([min(X), max(X)], [min(m*X+c), max(m*X+c)], color='red')
            plt.show()
            camera.snap()
        
        if ( abs(m_grad) < min_grad and abs(c_grad) < min_grad ):
            break
        
    return m, c

start_time = time.time()
m, c = gradient_descent_naive(X, y)
end_time = time.time()

print("%f seconds" %(end_time - start_time))

print("\n\nFinal:")
print("gdn_m=%f, gdn_c=%f" %(m, c) )
print("ls_m=%f, ls_c=%f" %(ls_m, ls_c) )

animation = camera.animate(interval=50, blit=True)
#animation.save('gradient descent.gif')

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.771
Method:                 Least Squares   F-statistic:                     306.8
Date:                Mon, 26 Oct 2020   Prob (F-statistic):           9.65e-31
Time:                        21:46:53   Log-Likelihood:                -312.21
No. Observations:                  92   AIC:                             628.4
Df Residuals:                      90   BIC:                             633.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         24.3479      2.390     10.186      0.0

In [10]:
#multiple linear regression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymysql
import time

def load_dbscore_data():
    conn = pymysql.connect(host = 'localhost',user = 'root',password = 'taeyoon!', db = 'datascienceassignment2')
    curs = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = "select * from db_score"
    curs.execute(sql)
    
    data  = curs.fetchall()
    
    curs.close()
    conn.close()
    
    X = [ ( t['attendance'],t['homework'],t['midterm'] ) for t in data ]
    X = np.array(X)
    
    y = [ (t['score']) for t in data]
    y = np.array(y)

    return X, y

X, y = load_dbscore_data()
X1, X2, X3 = np.hsplit(X, 3) #3개의 배열로 분리

import statsmodels.api as sm #Least Square
X_const = sm.add_constant(X)

model = sm.OLS(y, X_const)
ls = model.fit()

print(ls.summary())

ls_c = ls.params[0]
ls_m1 = ls.params[1]
ls_m2 = ls.params[2]
ls_m3 = ls.params[3]

def gradient_descent_vectorized(X, y):
    #epochs = 100000
    epochs = 2000000
    min_grad = 0.0001
    #learning_rate = 0.001
    learning_rate = 0.000001
    
    m1 = 0.0
    m2 = 0.0
    m3 = 0.0
    c = 0.0
    
    n = len(y)
    
    c_grad = 0.0
    m1_grad = 0.0
    m2_grad = 0.0
    m3_grad = 0.0

    
    for epoch in range(epochs):    
       
        y_pred = (m1*X1) + (m2*X2) + (m3*X3) + c #X를 통으로 집어넣는다
        m1_grad = (2*(y_pred - y)*X1).sum()/n # 한번에 계산
        m2_grad = (2*(y_pred - y)*X2).sum()/n # 한번에 계산
        m3_grad = (2*(y_pred - y)*X3).sum()/n # 한번에 계산
        c_grad = (2*(y_pred - y)).sum()/n
        
        m1 = m1 - learning_rate * m1_grad
        m2 = m2 - learning_rate * m2_grad
        m3 = m3 - learning_rate * m3_grad
        c = c - learning_rate * c_grad    
        
        if ( epoch % 1000 == 0):
            print("epoch %d: m1_grad=%f, m2_grad=%f, m3_grad=%f, c_grad=%f " %(epoch, m1_grad, m2_grad, m3_grad, c_grad) )
            print("m1=%f, m2=%f, m3=%f, c=%f \n" %(m1,m2,m3,c))
    
        if ( abs(m1_grad) < min_grad and abs(m2_grad) < min_grad and abs(m3_grad) < min_grad and  abs(c_grad) < min_grad ):
            break

    return m1,m2,m3,c

start_time = time.time()
m1,m2,m3, c = gradient_descent_vectorized(X, y)
end_time = time.time()

print("%f seconds" %(end_time - start_time))

print("\n\nFinal:")
print("gdv_m1=%f, gdv_m2=%f, gdv_m3=%f, gdv_c=%f" %(m1,m2,m3,c) )
print("ls_m1=%f, ls_m2=%f, ls_m3=%f, ls_c=%f" %(ls_m1,ls_m2,ls_m3, ls_c) )


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.817
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     131.2
Date:                Mon, 26 Oct 2020   Prob (F-statistic):           2.24e-32
Time:                        21:59:14   Log-Likelihood:                -302.25
No. Observations:                  92   AIC:                             612.5
Df Residuals:                      88   BIC:                             622.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -19.4116     15.824     -1.227      0.2

epoch 44000: m1_grad=2.290858, m2_grad=0.189062, m3_grad=-0.006626, c_grad=-21.300254 
m1=8.480847, m2=-0.196101, m3=-0.033580, c=2.120875 

epoch 45000: m1_grad=2.388131, m2_grad=0.148122, m3_grad=-0.007402, c_grad=-21.279392 
m1=8.478506, m2=-0.196269, m3=-0.033573, c=2.142164 

epoch 46000: m1_grad=2.469726, m2_grad=0.113717, m3_grad=-0.008054, c_grad=-21.260701 
m1=8.476076, m2=-0.196399, m3=-0.033565, c=2.163434 

epoch 47000: m1_grad=2.538145, m2_grad=0.084805, m3_grad=-0.008601, c_grad=-21.243835 
m1=8.473571, m2=-0.196498, m3=-0.033556, c=2.184686 

epoch 48000: m1_grad=2.595489, m2_grad=0.060510, m3_grad=-0.009060, c_grad=-21.228504 
m1=8.471004, m2=-0.196570, m3=-0.033548, c=2.205922 

epoch 49000: m1_grad=2.643526, m2_grad=0.040094, m3_grad=-0.009445, c_grad=-21.214464 
m1=8.468383, m2=-0.196620, m3=-0.033538, c=2.227144 

epoch 50000: m1_grad=2.683741, m2_grad=0.022939, m3_grad=-0.009768, c_grad=-21.201508 
m1=8.465719, m2=-0.196651, m3=-0.033529, c=2.248352 

epoch 51000: 

epoch 102000: m1_grad=2.849577, m2_grad=-0.066349, m3_grad=-0.011287, c_grad=-20.797639 
m1=8.317461, m2=-0.193690, m3=-0.032946, c=3.339692 

epoch 103000: m1_grad=2.848605, m2_grad=-0.066327, m3_grad=-0.011284, c_grad=-20.790514 
m1=8.314612, m2=-0.193624, m3=-0.032935, c=3.360486 

epoch 104000: m1_grad=2.847632, m2_grad=-0.066306, m3_grad=-0.011280, c_grad=-20.783392 
m1=8.311763, m2=-0.193558, m3=-0.032924, c=3.381273 

epoch 105000: m1_grad=2.846660, m2_grad=-0.066285, m3_grad=-0.011276, c_grad=-20.776272 
m1=8.308916, m2=-0.193491, m3=-0.032913, c=3.402053 

epoch 106000: m1_grad=2.845687, m2_grad=-0.066263, m3_grad=-0.011272, c_grad=-20.769155 
m1=8.306070, m2=-0.193425, m3=-0.032901, c=3.422826 

epoch 107000: m1_grad=2.844714, m2_grad=-0.066241, m3_grad=-0.011268, c_grad=-20.762040 
m1=8.303225, m2=-0.193359, m3=-0.032890, c=3.443591 

epoch 108000: m1_grad=2.843742, m2_grad=-0.066219, m3_grad=-0.011264, c_grad=-20.754928 
m1=8.300381, m2=-0.193293, m3=-0.032879, c=3.464350 


epoch 160000: m1_grad=2.793537, m2_grad=-0.065054, m3_grad=-0.011066, c_grad=-20.388443 
m1=8.153815, m2=-0.189880, m3=-0.032298, c=4.534049 

epoch 161000: m1_grad=2.792580, m2_grad=-0.065031, m3_grad=-0.011062, c_grad=-20.381459 
m1=8.151022, m2=-0.189814, m3=-0.032287, c=4.554434 

epoch 162000: m1_grad=2.791623, m2_grad=-0.065009, m3_grad=-0.011058, c_grad=-20.374477 
m1=8.148230, m2=-0.189749, m3=-0.032276, c=4.574812 

epoch 163000: m1_grad=2.790667, m2_grad=-0.064987, m3_grad=-0.011054, c_grad=-20.367498 
m1=8.145439, m2=-0.189684, m3=-0.032265, c=4.595183 

epoch 164000: m1_grad=2.789711, m2_grad=-0.064965, m3_grad=-0.011050, c_grad=-20.360521 
m1=8.142649, m2=-0.189619, m3=-0.032254, c=4.615547 

epoch 165000: m1_grad=2.788755, m2_grad=-0.064942, m3_grad=-0.011047, c_grad=-20.353547 
m1=8.139859, m2=-0.189555, m3=-0.032243, c=4.635904 

epoch 166000: m1_grad=2.787800, m2_grad=-0.064920, m3_grad=-0.011043, c_grad=-20.346575 
m1=8.137071, m2=-0.189490, m3=-0.032232, c=4.656254 


epoch 218000: m1_grad=2.738574, m2_grad=-0.063774, m3_grad=-0.010848, c_grad=-19.987301 
m1=7.993389, m2=-0.186144, m3=-0.031663, c=5.704907 

epoch 219000: m1_grad=2.737636, m2_grad=-0.063752, m3_grad=-0.010844, c_grad=-19.980454 
m1=7.990651, m2=-0.186080, m3=-0.031652, c=5.724891 

epoch 220000: m1_grad=2.736698, m2_grad=-0.063730, m3_grad=-0.010840, c_grad=-19.973610 
m1=7.987914, m2=-0.186016, m3=-0.031641, c=5.744868 

epoch 221000: m1_grad=2.735761, m2_grad=-0.063708, m3_grad=-0.010837, c_grad=-19.966768 
m1=7.985178, m2=-0.185952, m3=-0.031630, c=5.764838 

epoch 222000: m1_grad=2.734824, m2_grad=-0.063686, m3_grad=-0.010833, c_grad=-19.959929 
m1=7.982443, m2=-0.185889, m3=-0.031619, c=5.784801 

epoch 223000: m1_grad=2.733887, m2_grad=-0.063665, m3_grad=-0.010829, c_grad=-19.953092 
m1=7.979708, m2=-0.185825, m3=-0.031609, c=5.804758 

epoch 224000: m1_grad=2.732950, m2_grad=-0.063643, m3_grad=-0.010826, c_grad=-19.946257 
m1=7.976975, m2=-0.185761, m3=-0.031598, c=5.824707 


...

epoch 678000: m1_grad=2.339270, m2_grad=-0.054475, m3_grad=-0.009266, c_grad=-17.073011 
m1=6.827897, m2=-0.159003, m3=-0.027046, c=14.211178 

epoch 679000: m1_grad=2.338469, m2_grad=-0.054456, m3_grad=-0.009263, c_grad=-17.067163 
m1=6.825558, m2=-0.158948, m3=-0.027037, c=14.228248 

epoch 680000: m1_grad=2.337668, m2_grad=-0.054438, m3_grad=-0.009260, c_grad=-17.061317 
m1=6.823220, m2=-0.158894, m3=-0.027028, c=14.245313 

epoch 681000: m1_grad=2.336867, m2_grad=-0.054419, m3_grad=-0.009257, c_grad=-17.055472 
m1=6.820882, m2=-0.158839, m3=-0.027018, c=14.262371 

epoch 682000: m1_grad=2.336067, m2_grad=-0.054400, m3_grad=-0.009253, c_grad=-17.049630 
m1=6.818546, m2=-0.158785, m3=-0.027009, c=14.279424 

epoch 683000: m1_grad=2.335267, m2_grad=-0.054382, m3_grad=-0.009250, c_grad=-17.043790 
m1=6.816210, m2=-0.158730, m3=-0.027000, c=14.296470 

epoch 684000: m1_grad=2.334467, m2_grad=-0.054363, m3_grad=-0.009247, c_grad=-17.037951 
m1=6.813875, m2=-0.158676, m3=-0.026991, c=14.3

epoch 735000: m1_grad=2.294031, m2_grad=-0.053422, m3_grad=-0.009087, c_grad=-16.742836 
m1=6.695852, m2=-0.155928, m3=-0.026523, c=15.174899 

epoch 736000: m1_grad=2.293245, m2_grad=-0.053403, m3_grad=-0.009084, c_grad=-16.737100 
m1=6.693558, m2=-0.155874, m3=-0.026514, c=15.191639 

epoch 737000: m1_grad=2.292460, m2_grad=-0.053385, m3_grad=-0.009081, c_grad=-16.731367 
m1=6.691265, m2=-0.155821, m3=-0.026505, c=15.208373 

epoch 738000: m1_grad=2.291675, m2_grad=-0.053367, m3_grad=-0.009078, c_grad=-16.725636 
m1=6.688973, m2=-0.155767, m3=-0.026496, c=15.225102 

epoch 739000: m1_grad=2.290890, m2_grad=-0.053348, m3_grad=-0.009074, c_grad=-16.719907 
m1=6.686682, m2=-0.155714, m3=-0.026487, c=15.241825 

epoch 740000: m1_grad=2.290105, m2_grad=-0.053330, m3_grad=-0.009071, c_grad=-16.714179 
m1=6.684391, m2=-0.155661, m3=-0.026478, c=15.258542 

epoch 741000: m1_grad=2.289320, m2_grad=-0.053312, m3_grad=-0.009068, c_grad=-16.708454 
m1=6.682102, m2=-0.155607, m3=-0.026469, c=15.2

epoch 792000: m1_grad=2.249667, m2_grad=-0.052388, m3_grad=-0.008911, c_grad=-16.419046 
m1=6.566360, m2=-0.152912, m3=-0.026010, c=16.119983 

epoch 793000: m1_grad=2.248896, m2_grad=-0.052370, m3_grad=-0.008908, c_grad=-16.413421 
m1=6.564111, m2=-0.152860, m3=-0.026001, c=16.136399 

epoch 794000: m1_grad=2.248126, m2_grad=-0.052353, m3_grad=-0.008905, c_grad=-16.407799 
m1=6.561863, m2=-0.152807, m3=-0.025992, c=16.152809 

epoch 795000: m1_grad=2.247356, m2_grad=-0.052335, m3_grad=-0.008902, c_grad=-16.402178 
m1=6.559615, m2=-0.152755, m3=-0.025983, c=16.169214 

epoch 796000: m1_grad=2.246586, m2_grad=-0.052317, m3_grad=-0.008899, c_grad=-16.396560 
m1=6.557368, m2=-0.152703, m3=-0.025974, c=16.185614 

epoch 797000: m1_grad=2.245817, m2_grad=-0.052299, m3_grad=-0.008896, c_grad=-16.390943 
m1=6.555122, m2=-0.152650, m3=-0.025966, c=16.202008 

epoch 798000: m1_grad=2.245047, m2_grad=-0.052281, m3_grad=-0.008893, c_grad=-16.385329 
m1=6.552876, m2=-0.152598, m3=-0.025957, c=16.2

epoch 849000: m1_grad=2.206161, m2_grad=-0.051375, m3_grad=-0.008739, c_grad=-16.101517 
m1=6.439373, m2=-0.149955, m3=-0.025507, c=17.046789 

epoch 850000: m1_grad=2.205405, m2_grad=-0.051358, m3_grad=-0.008736, c_grad=-16.096002 
m1=6.437168, m2=-0.149904, m3=-0.025498, c=17.062888 

epoch 851000: m1_grad=2.204649, m2_grad=-0.051340, m3_grad=-0.008733, c_grad=-16.090488 
m1=6.434963, m2=-0.149852, m3=-0.025490, c=17.078981 

epoch 852000: m1_grad=2.203894, m2_grad=-0.051323, m3_grad=-0.008730, c_grad=-16.084976 
m1=6.432758, m2=-0.149801, m3=-0.025481, c=17.095069 

epoch 853000: m1_grad=2.203139, m2_grad=-0.051305, m3_grad=-0.008727, c_grad=-16.079466 
m1=6.430555, m2=-0.149750, m3=-0.025472, c=17.111151 

epoch 854000: m1_grad=2.202385, m2_grad=-0.051287, m3_grad=-0.008724, c_grad=-16.073958 
m1=6.428352, m2=-0.149698, m3=-0.025463, c=17.127228 

epoch 855000: m1_grad=2.201630, m2_grad=-0.051270, m3_grad=-0.008721, c_grad=-16.068452 
m1=6.426150, m2=-0.149647, m3=-0.025455, c=17.1

KeyboardInterrupt: 